In [1]:
import numpy as np

from skimage.io import imshow
from skimage.transform import resize

from sklearn.model_selection import train_test_split

from collections import namedtuple

import threading

from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Activation
from keras.optimizers import Adam

from augmentation import ImageGenerator
%matplotlib inline

Using TensorFlow backend.


In [2]:
def get_train_data(path='public_data/00_input/train/'):
    from skimage.io import imread
    with open(path + 'gt.csv') as navigation_file:
        splits = [line.split(',') for line in navigation_file]
        points_number = int((len(splits[0]) - 1) / 2)
        train_x = np.array([imread(path + 'images/' + split[0]) for split in splits[1:]])
        train_y = np.array([[int(x) for x in split[1:]] for split in splits[1:]])
        return train_x, train_y, points_number

def get_colored_data(images, coordinates):
    return (
        np.array([image for image in images if len(np.shape(image)) == 3]),
        np.array([answer for image, answer in zip(images, coordinates) if len(np.shape(image)) == 3])
    )

def get_resized_train_data(train_x, train_y, shape=(100, 100, 3)):
    resized_x = np.array([resize(image, shape, mode='reflect') for image in train_x])
    resized_y = np.array([[
            int(coordinate * shape[i % 2] / np.shape(image)[i % 2])
            for i, coordinate in enumerate(answer)
        ] for image, answer in zip(train_x, train_y)
    ])
    return np.array(resized_x), np.array(resized_y)

In [3]:
all_x, all_y, points_number = get_train_data()
colored_x, colored_y = get_colored_data(all_x, all_y)
resized_x, resized_y = get_resized_train_data(colored_x, colored_y)
input_shape=np.shape(resized_x[0])
train_x, test_x, train_y, test_y = train_test_split(
    resized_x, resized_y, test_size=0.3, random_state=42
)

In [4]:
datagen = ImageGenerator()

In [5]:
regularization_lambda = 5e-6

def add_Conv2D_relu(model, n_filter, filters_size, input_shape=None):
    if input_shape is not None:
        model.add(Conv2D(
            n_filter, filters_size, padding='same',
            kernel_regularizer=regularizers.l2(regularization_lambda),
            activation='elu', input_shape=input_shape,
        ))
    else:
        model.add(Conv2D(
            n_filter, filters_size, padding='same',
            kernel_regularizer=regularizers.l2(regularization_lambda),
            activation='elu'
        ))

model = Sequential()

add_Conv2D_relu(model, 64, (3, 3), input_shape)
add_Conv2D_relu(model, 64, (3, 3))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())

add_Conv2D_relu(model, 128, (3, 3))
add_Conv2D_relu(model, 128, (3, 3))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())

add_Conv2D_relu(model, 256, (3, 3))
add_Conv2D_relu(model, 256, (3, 3))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())

model.add(Flatten(name='flatten'))
model.add(Dense(512, activation='elu', kernel_regularizer=regularizers.l2(regularization_lambda)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='elu', kernel_regularizer=regularizers.l2(regularization_lambda)))

model.add(Dense(points_number * 2))

In [6]:
model.compile(loss='mean_squared_error',
              optimizer=Adam(),
              metrics=['mse'])

In [7]:
class MyClallback:
    def __init__(self, model, directory):
        self._model = model
        self._directory = directory
        self._counter = 0

    def set_model(self, model):
        pass

    def set_params(self, params):
        pass

    def on_train_begin(self, logs):
        pass

    def on_train_end(self, logs):
        pass

    def on_epoch_begin(self, epoch, logs):
        pass

    def on_epoch_end(self, epoch, logs):
        self._model.save(
            self._directory + '/' + str(self._counter) +
            '_mse:_' + str(logs['mean_squared_error']) +
            '_val_mse:_' + str(logs['val_mean_squared_error']) +
            '.npz'
        )
        self._counter += 1

    def on_batch_begin(self, batch, logs):
        pass

    def on_batch_end(self, batch, logs):
        pass

In [ ]:
model.fit_generator(datagen.flow(train_x, train_y, batch_size=32),
                    steps_per_epoch=int(len(train_y) / 32), epochs=1000,
                    callbacks=[MyClallback(model, 'aug_models')], validation_data=(test_x, test_y))

Epoch 1/1000
117/130 [==========================>...] - ETA: 4s - loss: 350.4289 - mean_squared_error: 350.4148

74.1453 validation